# Defining custom problems, optimizers and solvers

## User defined class

Big advantage of using QHyper is the ability to run experiments from configuration file.
But this only allows to use predefined problems, optimizers and solvers.
To overcome this limitation, QHyper will try to import any [Problem](../../problems.rst), [Optimizer](../../optimizers.rst) or [Solver](../../solvers.rst) class from directory `custom` or `QHyper/custom`.
It is required that these classes inherit from one of them and implement required methods. The class will be available by its attribute name if provided or by its class name in lower case.

.. note::
    Below is a simple example of how to define custom problem, but the same applies to optimizers and solvers. We choose to show problem because it is the most complex of the three.

## Creating a custom problem

Assume we want to  minimize $\underbrace{2x_0 + 5x_1 + x_0x_1}_{cost function}$ subject to $\underbrace{x_0 + x_1 = 1}_{constraint\ eq}$ and $\underbrace{5x_0 + 2x_1 \leq 5}_{constraint\ le}$

In QHyper, every problem needs to be a subclass of the [Problem class](../../problems.rst).

In general, the cost function and every constraint should be expressed as dict-based [Polynomial](../../polynomial.rst)s, but usually it is easier to initially express them in a more user-friendly format (such as SymPy syntax), and then convert it them into Polynomials. A Polynomial is comprised of a dictionary where the keys are tuples containing variables, and the values represent their coefficients.

To define the constraints, the [Constraint](../../constraint.rst) class is used. Each constraint involves Polynomials on the left-hand side (lhs) and right-hand side (rhs), a comparison operator, and optional data such as a method for handling inequalities, a label, and a group identifier.

`Note:` QHyper always assumes that the objective is to **minimize** the cost function.

### Using SymPy syntax

In [4]:
import sympy

from QHyper.problems.base import Problem
from QHyper.constraint import Constraint, Operator, UNBALANCED_PENALIZATION
from QHyper.polynomial import Polynomial
from QHyper.parser import from_sympy


class SimpleProblemSympy(Problem):

    def __init__(self) -> None:

        # Define the necessary SymPy variables
        self.x0, self.x1 = sympy.symbols('x0 x1')

        self.objective_function: Polynomial = self._get_objective_function()
        self.constraints: list[Constraint] = self._get_constraints()

    def _get_objective_function(self) -> Polynomial:
        # Define the cost function
        cost = 2*self.x0 + 5*self.x1 + self.x0*self.x1

        # Return the cost function parsed into a Polynomial
        return from_sympy(cost)

    def _get_constraints(self) -> Polynomial:
        constraints: list[Constraint] = []

        # Define the left-hand side of the equality constraint
        constraint_eq_lhs = self.x0 + self.x1
        # Define the right-hand side of the equality constraint
        constraint_eq_rhs = 1
        # Add a new equality constraint, the lhs needs to be parsed from the SymPy syntax
        # the rhs which is a number can be casted to the Polynomial
        constraints.append(Constraint(from_sympy(constraint_eq_lhs),
                                            Polynomial(constraint_eq_rhs)))


        constraint_le_lhs = 5 * self.x0 + 2 * self.x1
        constraint_le_rhs = 5
        # Add a new inequality constraint, apart from the lhs, and rhs
        # specify the comparison operator and the method for handling the inequality
        constraints.append(Constraint(from_sympy(constraint_le_lhs),
                                            Polynomial(constraint_le_rhs),
                                            Operator.LE,
                                            UNBALANCED_PENALIZATION))

        return constraints

    def get_score(self, result: str, penalty:float = 0) -> float:
        """
        This function is used to evaluate the quality of the result (business value).
        """
        # Convert the binary result to a list of integers
        x = [int(val) for val in result]

        # If the constraints are satisfied return the value of the objective function
        if x[0] + x[1] == 1 and 5 * x[0] + 2 * x[1] <= 5:
            return 2 * x[0] + 5 * x[1] + x[0] * x[1]

        # Otherwise return some arbitrary penalty
        return penalty

### Using Dict syntax

In [17]:
from QHyper.problems.base import Problem
from QHyper.constraint import Constraint, Operator, UNBALANCED_PENALIZATION
from QHyper.polynomial import Polynomial


class SimpleProblemDicts(Problem):

    def __init__(self) -> None:
        self.objective_function: Polynomial = self._get_objective_function()
        self.constraints: list[Constraint] = self._get_constraints()

    def _get_objective_function(self) -> Polynomial:
        # Express the cost function as a dict
        # the keys are tuples containing variables, and the values represent the coefficients
        cost = {('x0', 'x1'): 1.0, ('x0',): 2.0, ('x1',): 5.0}
        # Cast the cost dict to a Polynomial
        return Polynomial(cost)

    def _get_constraints(self) -> None:
        constraints: list[Constraint] = []

        # Define the left-hand side of the equality constraint
        constraint_eq_lhs = {('x0',): 1.0, ('x1',): 1.0}
        # Define the right-hand side of the equality constraint
        constraint_eq_rhs = {(): 1}
        # Add a new equality constraint casting both the lhs and rhs to Polynomials
        constraints.append(Constraint(Polynomial(constraint_eq_lhs),
                                            Polynomial(constraint_eq_rhs)))

        constraint_le_lhs = {('x0',): 5.0, ('x1',): 2.0}
        constraint_le_rhs = {(): 5}
        # Add a new equality constraint, apart from the lhs, and rhs
        # specify the comparison operator and the method for handling the inequality
        constraints.append(Constraint(Polynomial(constraint_le_lhs),
                                            Polynomial(constraint_le_rhs),
                                            Operator.LE,
                                            UNBALANCED_PENALIZATION))

        return constraints

    def get_score(self, result: str, penalty:float = 0) -> float:
        """
        This function is used to evaluate the quality of the result (business value).
        """
        # Convert the binary result to a list of integers
        x = [int(val) for val in result]

        # If the constraints are satisfied return the value of the objective function
        if x[0] + x[1] == 1 and 5 * x[0] + 2 * x[1] <= 5:
            return 2 * x[0] + 5 * x[1] + x[0] * x[1]

        # Otherwise return some arbitrary penalty
        return penalty

To learn more about different solvers check out this [tutorial](../solver_configuration.rst).